import tweepy
import json

In [202]:
import pyodbc
from __future__ import absolute_import, print_function
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from sqlalchemy.exc import ProgrammingError
import tweepy
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import math, re, string, requests, json



In [222]:
conn = pyodbc.connect(r'DSN=starbucks')
curr= conn.cursor()
analyzer = SentimentIntensityAnalyzer()


language_codes = ["fr", "de", "es", "it", "ru", "ja", "ar", "zh"]


apikey = 'J6i0D62a4Mw5Ah7H2Yll56Juv'
apisecret ='rQUI6gGOLVjn2KrDNDYBjvkmQbLKjnQsDuqmQ09pYln74U19sU'
accesstoken = '887695309610135552-9iEwdo4RYjRDHkeImgvQLuvKFiPhAcE'
tokensecret = 'zXY5Wq4cs3D2hjD5DIpTit2k7k1VbmSIC8KWzrpbtCAIo'


In [228]:
class StdOutListener(StreamListener):
    def on_status(self, status):
    
       try:
            if status.user.lang in language_codes:
                api_url = "http://mymemory.translated.net/api/get?q={}&langpair={}|{}".format(status.text, status.user.lang, "en")
                hdrs ={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'}
                response = requests.get(api_url, headers=hdrs)
                response_json = json.loads(response.text)
                translation = response_json["responseData"]["translatedText"]
                translator_name = "MemoryNet Translation Service"
                vs = analyzer.polarity_scores(translation)
                sent = vs['compound']
                print('noneng')
            else:
                vs = analyzer.polarity_scores(status.text)
                sent = vs['compound']
                translation = status.text
                print('eng')
        except:
            return
        status.text = str.replace(status.text, '\'', '\'\'')
        q= 'insert into starbucks(\
        id_str, created,Uk_time, text, fav,name, followers, sentiment, language)\
        values (\'%s\',\'%s\', getdate(),\'%s\',\'%s\',\'%s\',\'%s\', \'%s\',\'%s\,\'%s\')'\
        %(status.id_str, status.created_at, status.text, status.favorite_count, status.user.screen_name,\
          status.user.followers_count, sent, status.user.lang, translation)
        
     
        try:
            curr.execute(q)
            conn.commit()
            
        except:
            print(q)
        #('insert into starbucks(\
        #id_str, created,text, fav,name, description, loc, user_created, follower)\
        #values (%s,%s,%s,%s,%s,%s,%s,%s,%s)'\
        #%(status.id_str, status.created_at, status.text, status.favorite_count, status.user.screen_name, status.user.description, status.user.location, status.user.created_at, status.user.followers_count))
        
        
         
               
    def on_error(self, status_code):
        if status_code == 420:
            return False



In [229]:
l = StdOutListener()
auth = tweepy.OAuthHandler(apikey, apisecret)
auth.set_access_token(accesstoken, tokensecret)
api = tweepy.API(auth)
stream = Stream(auth, l)
stream.filter(track=['starbucks'])

eng
insert into starbucks(        id_str, created,Uk_time, text, fav,name, followers, sentiment, language)        values ('889841732375937024','2017-07-25 13:36:16', getdate(),'Being able to just run to Dunkin or Starbucks whenever I need/want to during the day makes me love my job even more💝','0','xallllyssa','224', '0.6369','en\,'Being able to just run to Dunkin or Starbucks whenever I need/want to during the day makes me love my job even more💝')
eng
insert into starbucks(        id_str, created,Uk_time, text, fav,name, followers, sentiment, language)        values ('889841752521285632','2017-07-25 13:36:20', getdate(),'@mrBobbyBones we need a Starbucks Steven shirt. "Starbucks Steven is my barista"','0','ncsu_blowery','412', '0.0','en\,'@mrBobbyBones we need a Starbucks Steven shirt. "Starbucks Steven is my barista"')
noneng
insert into starbucks(        id_str, created,Uk_time, text, fav,name, followers, sentiment, language)        values ('889841768883204096','2017-07-25 13:36:24'

AttributeError: 'NoneType' object has no attribute 'encode'